In [1]:
# Import dependencies
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re # to work with regex
import time # to add delays between requests


## Scraping one page

In [2]:
# Launch browser
driver = webdriver.Chrome()
# Visit listing.ca
url = "https://toronto.listing.ca/"
driver.get(url)

In [3]:
# Create a BeautifulSoup object from the scraped HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
# List of listings in the html soup, found using class sl
listings_html = soup.find_all('div', 'sl')
# Print first element of the list
listings_html[0]


<div class="sl">
<div class="slt">
<div class="slt_address"><a href="https://toronto.listing.ca/286-main-st-911.E8018446.htm#15-1" style="font-size:16px;">286 Main St 911</a></div>
<div class="slt_price">$619,900 CAD</div>
<div class="slt_baths"><span class="slt_bb_value">1</span> baths</div>
<div class="slt_beds"><span class="slt_bb_value">1+1</span> beds</div>
</div>
<div class="sl_loc">
<div>
<na2><a class="na2" href="https://toronto.listing.ca">Toronto</a></na2>
<na2><a class="na2" href="https://toronto.listing.ca/east-end-danforth.htm">East End-Danforth</a></na2>
<na2><a class="na2" href="https://toronto.listing.ca/main-st.htm">Main St</a></na2>
</div>
<div>
<div>( <script type="text/javascript">document.write('<a href="https://listing.ca/buy-real-estate-with-bitcoin.htm" target="_blank">?</a>');</script><a href="https://listing.ca/buy-real-estate-with-bitcoin.htm" target="_blank">?</a> )</div>
<div>- BTC</div>
</div>
</div>
<div style="font-size:12px;">Assignment sale at the BRAN

In [5]:
# Loop through listings and append the clean info a list of dictionaries
listings_list = []
for item in listings_html:
    # Create a dictionary containing the info for a listing
    listing = {
        'url': item.find('div', 'slt_address').a['href'],
        'price': item.find('div', 'slt_price').text.strip('$ CAD').replace(',', ''),
        'baths': item.find('div', 'slt_baths').text.rstrip(' baths'),
        'beds': item.find('div', 'slt_beds').text.rstrip(' beds').partition('+')[0],
        'dens': item.find('div', 'slt_beds').text.rstrip(' beds').partition('+')[2],
        'street': item.find_all('na2')[2].text,
        'neighbourhood': item.find_all('na2')[1].text
    }
    # Extract mls_id from url
    listing['mls_id'] = re.search('[A-Z]\\d{7}', listing['url']).group()
    # Append to the list
    listings_list.append(listing)
# Check results
listings_list[0:3]

[{'url': 'https://toronto.listing.ca/286-main-st-911.E8018446.htm#15-1',
  'address': '286 Main St 911',
  'price': '$619,900 CAD',
  'baths': '1 baths',
  'beds': '1+1 beds',
  'neighbourhood': 'East End-Danforth',
  'mls_id': 'E8018446'},
 {'url': 'https://toronto.listing.ca/18-willowlea-dr.E7353624.htm#15-2',
  'address': '18 Willowlea Dr',
  'price': '$1,699,900 CAD',
  'baths': '0 baths',
  'beds': '0 beds',
  'neighbourhood': 'Highland Creek',
  'mls_id': 'E7353624'},
 {'url': 'https://toronto.listing.ca/5-kingsbury-cres.E7292902.htm#15-3',
  'address': '5 Kingsbury Cres',
  'price': '$15,000,000 CAD',
  'baths': '1 baths',
  'beds': '2 beds',
  'neighbourhood': 'Birchcliffe-Cliffside',
  'mls_id': 'E7292902'}]

In [6]:
# Create dataframe with info from first page's listings
df = pd.DataFrame(listings_list)
df.head()

,url,address,price,baths,beds,neighbourhood,mls_id
0,https://toronto.listing.ca/286-main-st-911.E80...,286 Main St 911,"$619,900 CAD",1 baths,1+1 beds,East End-Danforth,E8018446
1,https://toronto.listing.ca/18-willowlea-dr.E73...,18 Willowlea Dr,"$1,699,900 CAD",0 baths,0 beds,Highland Creek,E7353624
2,https://toronto.listing.ca/5-kingsbury-cres.E7...,5 Kingsbury Cres,"$15,000,000 CAD",1 baths,2 beds,Birchcliffe-Cliffside,E7292902
3,https://toronto.listing.ca/215-queen-st-606.C7...,215 Queen St 606,"$529,000 CAD",1 baths,1 beds,Waterfront Communities C1,C7266728
4,https://toronto.listing.ca/10-park-lawn-rd-140...,10 Park Lawn Rd 1408,"$624,900 CAD",1 baths,1+1 beds,Mimico,W7239426


In [8]:
# Quit browsing session
driver.quit()

## Function and page loops

In [5]:
# Define function to scrape a single page,
# @arg html: scraped html
# @return list of listing dictionaries
def get_listing_info(html):
    # Create a BeautifulSoup object from the scraped HTML
    soup = BeautifulSoup(html, 'html.parser')
    # List of listings in the html soup, found using class sl
    listings_html = soup.find_all('div', 'sl')
    # Empty list to store listing dicts
    listings_list = []
    for item in listings_html:
        # Create a dictionary containing the info for a listing
        listing = {
            'url': item.find('div', 'slt_address').a['href'],
            'address': item.find('div', 'slt_address').text,
            'price': item.find('div', 'slt_price').text.strip('$ CAD').replace(',', ''),
            'baths': item.find('div', 'slt_baths').text.rstrip(' baths'),
            'beds': item.find('div', 'slt_beds').text.rstrip(' beds').partition('+')[0],
            'dens': item.find('div', 'slt_beds').text.rstrip(' beds').partition('+')[2],
            'street': item.find_all('na2')[2].text,
            'neighbourhood': item.find_all('na2')[1].text
        }
        # if no den, put 0
        listing['dens'] = '0' if listing['dens'] == '' else listing['dens']
        # Extract mls_id from url
        listing['mls_id'] = re.search('[A-Z]\\d{7}', listing['url']).group()
        # Append to the list
        listings_list.append(listing)
    # Return expanded listings_list
    return listings_list

In [9]:
# Define function that scrapes a range of pages from listing.ca toronto section
# @arg property_type: 1 is condos, 2 freehold townhouses, 3 condo townhouses, 4 no filter, 5 detached houses
# @return dataframe of listing data
def get_df_from_pages(start_page, end_page, property_type):
    # Empty listing info list
    listing_info_list = []
    # Launch browser
    driver = webdriver.Chrome()
    # Loop tthrough a range corresponding to search pages and extend listing info list
    for i in range(start_page, end_page + 1):
        url = f'https://listing.ca/mls/?{property_type}.cy.........{i}..$'
        driver.get(url)
        listing_info_list.extend(get_listing_info(driver.page_source))
        # Wait 1 second, to avoid connection errors
        time.sleep(1)
    # Quit browsing session
    driver.quit()
    # Make df with list of dictionaries
    df = pd.DataFrame(listing_info_list)
    # Put property type in column
    df['property_type_code'] = f'ptc_{property_type}'
    # Change type of these columns to integer
    for col_name in ['price', 'baths', 'beds', 'dens']:
        df[col_name] = df[col_name].astype('int64')
    return df

In [10]:
# Test function using pages 2 to 3, should result in 40 records (20 per page)
test_df = get_df_from_pages(2, 3, 1)
print(test_df.info())
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   url                 40 non-null     object
 1   address             40 non-null     object
 2   price               40 non-null     int64 
 3   baths               40 non-null     int64 
 4   beds                40 non-null     int64 
 5   dens                40 non-null     int64 
 6   street              40 non-null     object
 7   neighbourhood       40 non-null     object
 8   mls_id              40 non-null     object
 9   property_type_code  40 non-null     object
dtypes: int64(4), object(6)
memory usage: 3.2+ KB
None


,url,address,price,baths,beds,dens,street,neighbourhood,mls_id,property_type_code
0,https://toronto.listing.ca/2799-kingston-rd-32...,2799 Kingston Rd 321,529000,1,1,1,Kingston Rd,Cliffcrest,E8029328,ptc_1
1,https://toronto.listing.ca/400-mclevin-ave-151...,400 Mclevin Ave 1510,559800,2,2,0,Mclevin Ave,Malvern,E8029156,ptc_1
2,https://toronto.listing.ca/8-park-rd-3710.C803...,8 Park Rd 3710,1150000,2,2,0,Park Rd,Rosedale-Moore Park,C8030002,ptc_1
3,https://toronto.listing.ca/33-frederick-todd-w...,33 Frederick Todd Way 1108,1299000,3,2,1,Frederick Todd Way,Thorncliffe Park,C8029996,ptc_1
4,https://toronto.listing.ca/608-richmond-st-100...,608 Richmond St 1001,665000,1,1,1,Richmond St,Waterfront Communities C1,C8029936,ptc_1
